# 🌍 LangGraph Example: Travel Assistant with Weather & Historical Fact

This notebook demonstrates a LangGraph-based multi-agent assistant that:

- Accepts a user query (e.g., "I'm going to Paris")
- Extracts a city name
- Fetches weather data from Open-Meteo API
- Retrieves a fact from Wikipedia
- Summarizes both using an LLM

It uses real external APIs and demonstrates LangGraph’s modular workflow structure.

In [11]:
# ✅ Install required packages
!pip install -q langchain openai langgraph requests

In [23]:
import os
from dotenv import load_dotenv
import json

# Load environment variables from .env file
load_dotenv()

# Check that the key is loaded
assert os.getenv("OPENAI_API_KEY") is not None, "OPENAI_API_KEY is not set"

from langchain_openai import ChatOpenAI

# Initialize LLM using env key
llm = ChatOpenAI(temperature=0.3)

In [13]:
# ✅ Define state schema
from typing import TypedDict

class State(TypedDict):
    user_query: str
    city: str
    weather: str
    fact: str
    final_summary: str

In [14]:
# 🌐 External API tools
import requests

def get_weather(city: str) -> str:
    geo = requests.get(f"https://nominatim.openstreetmap.org/search?format=json&q={city}",
                       headers={"User-Agent": "langchain-agent"}).json()
    lat, lon = geo[0]["lat"], geo[0]["lon"]
    weather = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current_weather=true"
    ).json()
    temp = weather["current_weather"]["temperature"]
    return f"The current temperature in {city} is {temp}°C."

def get_fact(city: str) -> str:
    city_url = city.replace(" ", "_")
    r = requests.get(
        f"https://en.wikipedia.org/api/rest_v1/page/summary/{city_url}"
    )
    if r.status_code != 200:
        return f"No Wikipedia summary found for {city}"
    return r.json().get("extract", "No summary available.")

In [27]:
# 🤖 Node functions

def extract_city(state: State) -> dict:
    user_query = state["user_query"]
    if not user_query:
        raise ValueError("User query is required.")
    
    prompt = f"""
    You are a helpful assistant who can understand complex and 
    ill-formed queries. Return the following two pieces of
    information "city" and "fact" from {user_query}.
    1. Extract the city or town name from the query: 
    If no city or town is mentioned, return "Unknown". 
    If more than one city is mentioned, return the first one.
    2. Extract a question about the city from the query. Examples
    of a question are: "What museums are open?" and "What is a fun fact?"

    Return a JSON object like this:
    {{
    "city": "<city name or Unknown>",
    "fact": "<question about the city or Unknown>"
    }}

    Rules:
    - If no city is found, use "Unknown"
    - If no question is found, use "Unknown"
    
    """

    # Send prompt to LLM
    response = llm.invoke(prompt).content.strip()

    try:
        parsed = json.loads(response)
    except json.JSONDecodeError:
        raise ValueError(f"Could not parse response: {response}")

    city = parsed.get("city", "Unknown")
    fact = parsed.get("fact", "Unknown")
    print(f"Extracted city: {city}, fact: {fact}")
    return {**state, "city": city, "fact": fact}

def call_weather_tool(state: State) -> dict:
    weather = get_weather(state["city"])
    return {"weather": weather}

def call_fact_tool(state: State) -> dict:
    fact = get_fact(state["city"])
    return {"fact": fact}

def summarize(state: State) -> State:
    prompt = f"""You are a helpful assistant.
You were asked: '{state['user_query']}'

Here is the weather report: {state['weather']}
Here is a historical fact: {state['fact']}

Give a friendly summary in 1-2 sentences."""
    response = llm.invoke(prompt)
    return {**state, "final_summary": response.content}

In [28]:
# 🧩 Build the LangGraph
from langchain_core.runnables import RunnableLambda
from langgraph.graph import StateGraph, END

builder = StateGraph(State)
builder.add_node("extract_city", RunnableLambda(extract_city))
builder.add_node("fetch_weather", RunnableLambda(call_weather_tool))
builder.add_node("fetch_fact", RunnableLambda(call_fact_tool))
builder.add_node("summarize_all", RunnableLambda(summarize))

builder.set_entry_point("extract_city")
builder.add_edge("extract_city", "fetch_weather")
builder.add_edge("fetch_weather", "fetch_fact")
builder.add_edge("fetch_fact", "summarize_all")
builder.add_edge("summarize_all", END)

graph = builder.compile()

In [31]:
# 🚀 Run the assistant
user_input = "What is the temperature in Pasadena? What universities are in the city?"
initial_state = {"user_query": user_input}

result = graph.invoke(initial_state)
print(result["final_summary"])

Extracted city: Pasadena, fact: What universities are in the city?
The current temperature in Pasadena is 26.1°C. Pasadena is a vibrant city in Los Angeles County, known for its cultural significance and historical charm.
